# RAG FAQ-app
 

### Step 1: Load the Data

In [12]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader('dockuno.csv')
dataset = loader.load()

### Step 2: Load embeddings

In [19]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# myembedmodels = OpenAIEmbeddings(openai_api_key="")

from langchain_community.embeddings import OllamaEmbeddings
myembedmodels = OllamaEmbeddings(model="all-minilm")

C:\Users\saura\AppData\Local\Temp\ipykernel_18340\4073483054.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  myembedmodels = OllamaEmbeddings(model="all-minilm")


### Step 3: store embeddings into Vector Database

In [22]:
# pip install -qU langchain-community faiss-cpu
from langchain.vectorstores import FAISS
db = FAISS.from_documents( documents=dataset , embedding=myembedmodels)

In [32]:
# db.similarity_search("what is the fees in dollars?")

### Step 4: Prepare template prompt


In [30]:
dockuno_template = """
You are a world class business development representative for Dockuno program at eduLearn.ai .
I will share a prospect's message with you and you will give me the best answer that I should send to this prospect based on past best practices, and you will follow ALL of the rules below:
1/ Response should be very similar or even identical to the past best practies, in terms of length, ton of voice, logical arguments and other details
2/ If the best practice are irrelevant, then try to mimic the style of the best practice to prospect's message

Below is a message I received from the prospect:
{message}

Here is a list of best practies of how we normally respond to prospect in similar scenarios:
{best_practice}

Please write the best response that I should send to this prospect:
"""
from langchain.prompts import PromptTemplate

myprompt = PromptTemplate(template=dockuno_template, input_variables=["message", "best_practice"])

### Step 5 : prepare llm model

In [28]:
from langchain.chat_models import ollama
myllm = ollama.ChatOllama(model="tinyllama", temperature=0)

C:\Users\saura\AppData\Local\Temp\ipykernel_18340\1571441268.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  myllm = ollama.ChatOllama(model="tinyllama", temperature=0)


### prepare chain

In [33]:
from langchain.chains import LLMChain

my_chain = LLMChain(llm=myllm , prompt=myprompt)
best_practice = db.similarity_search("what is the fees in dollars?")
response = my_chain.run(message="what is the fees of dockuno in dollar", best_practice = best_practice)

C:\Users\saura\AppData\Local\Temp\ipykernel_18340\4143750790.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = my_chain.run(message="what is the fees of dockuno in dollar", best_practice = best_practice)


In [34]:
print(response)

Dear [Prospect],

Thank you for reaching out to us regarding your interest in our Dockuno program. We are excited to provide you with more information about our course structure and pricing.

Our course is designed to cover a variety of Docker concepts, including virtualization, containers, networking, and orchestration. The modules are structured into video lectures, hands-on labs, quizzes, and final project assessments. We believe that this approach provides you with a comprehensive understanding of Docker technology, as well as practical experience in using it.

Our course fee is [insert price], which includes all materials and support throughout the program. Our pricing is competitive compared to other similar courses on the market.

We understand that time is valuable, so we offer self-paced learners the option to complete the course at their own convenience. We also provide live Q&A sessions periodically where you can ask questions directly to our instructors. These sessions help